<a href="https://colab.research.google.com/github/MIchaelDFernandes/imersao-ia-alura-google/blob/main/imersao_ia_alura_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scriptizinho Python de última hora para o projeto Imersão IA - Alura & Google

**Instalando o SDK do google e importando bibliotecas**

In [ ]:
!pip install -q -U google-generativeai



In [48]:
import google.generativeai as genai
import pandas as pd
import os
from google.colab import userdata
import time


**Configurando o modelo**

Aqui começou minha dor de cabeça.
O **Gemini 1.0** não funciou com alguns dos arquivos.
Daí troquei para o **gemini-1.5-pro-latest**. Que por sua vez, responde de forma diferente, então tive que ajustar as respostas

In [114]:
genai.configure(api_key=userdata.get('API-KEY'))

# Criando o modelo
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")



### Funções

Função para obter uma resposta do Gemini

In [113]:
def obter_resposta_gemini(nomes: pd.Series) -> pd.Series:
  nomes_str = ", ".join(nomes.tolist())
  prompt = f"""
  PERGUNTA: Ana Carolina Silva, Lucas Oliveira Santos, Pedro Henrique Souza, Isabela Pereira Costa
  RESPOSTA: feminino, masculino, masculino, feminino

  PERGUNTA: {nomes_str}
  RESPOSTA:
  """

  response = model.generate_content(prompt)

  #Ajuste na resposta:
  #A palavra 'resposta' aparece na response
  # Aparece uma qubra de linha no final da response
  return pd.Series(response.text[10:].strip().split(", "))


Função para ler um arquivo .csv e salvar no formato do Excel com uma coluna *Gênero* adiconada

In [121]:
def exportar_para_excel(nome_arq: str):
  df = pd.read_csv(nome_arq, delimiter=";", skiprows=2, encoding="utf-8")
  resposta = obter_resposta_gemini(df["Nome do Aluno"])
  df["Gênero"] = resposta

  # Organizando as colunas
  df = df[
    ["Série", "Nº", "Nome do Aluno", "Gênero", "RA",
     "Dig. RA", "Data de Nascimento", "Data Início Matrícula",
     "Data Fim Matrícula", "Situação",
     "Endereço do Aluno"]]

  # Exporta para o formato do Excel no mesmo diretório
  df.to_excel(f"{nome_arq[:2]}-com-generos.xlsx", index=False)



### Manipulações

**Obtendo os arquivos .csv necessários do diretório do script.**

Que por sinal são meio estranhos, saca só o endereço desses alunos e a UF 😅:

In [130]:
df = pd.read_csv("1A.csv", delimiter=";", skiprows=2, encoding="utf-8")
df[["UF RA","Endereço do Aluno"]]

,UF RA,Endereço do Aluno
0,GoT,Braavos
1,GoT,Lorath
2,GoT,Muralha (The Wall)
3,GoT,Muralha (The Wall)
4,GoT,Norvos
5,GoT,Braavos
6,GoT,Winterfell
7,GoT,Rocha do Dragão (Dragonstone)
8,GoT,Pentos
9,GoT,Jardim de Cima (Highgarden)


In [ ]:
dir_list = os.listdir(".")
arquivos_csv = []

for i in dir_list:
    if i.endswith(".csv"):
        arquivos_csv.append(i)

**Convertendo cada arquivo .csv encontrado para o formato Excel com uma coluna "Gênero" adicionada**

Aqui também quebrei a cabeça, recomenda buscar um café ☕, pois parece que o Google tem uma restrição de 2 requests por minuto, o que me fez adicionar um *sleep de 30 segundos* 😞

In [128]:
for arq in arquivos_csv:
  exportar_para_excel(arq)
  time.sleep(30)


Abrindo um dos arquivos para checagem

In [131]:
df_alunos = pd.read_excel("1A-com-generos.xlsx", usecols="C,D")
df_alunos

,Nome do Aluno,Gênero
0,ALANNA LIMA PEREIRA,feminino
1,ANA VALENTINA DOS SANTOS NASCIMENTO,feminino
2,BEATRIZ DOS SANTOS RAMOS,feminino
3,BRAYAN DOS SANTOS,masculino
4,BRAYAN JUVÊNIO MONTEIRO DUARTE,masculino
5,BRYAN LUCAS BRITO GOMES,masculino
6,DAVI HENRY DE SOUZA FARIAS,masculino
7,DAVI OTÁVIO RODRIGUES OLIVEIRA,masculino
8,DAVID GABRIELL DA SILVA,masculino
9,ESTHER DA SILVA BRITO,feminino
